In [136]:
from xbos import get_client
from xbos.services.hod import HodClient
from xbos.devices.thermostat import Thermostat

import datetime
import math
import sys
import threading
import time
import traceback

import pytz
import yaml


In [137]:
    # Start of FIX for missing Brick query
    thermostat_query = """SELECT ?zone ?uri FROM  %s WHERE {
              ?tstat rdf:type brick:Thermostat .
              ?tstat bf:controls ?RTU .
              ?RTU rdf:type brick:RTU .
              ?RTU bf:feeds ?zone. 
              ?zone rdf:type brick:HVAC_Zone .
              ?tstat bf:uri ?uri.
              };"""

In [138]:
client = get_client()
hc = HodClient("xbos/hod", client)

Saw [xbos/hod] HodDB 8 seconds 315.821 ms ago


In [139]:
BUILDING = "avenal-veterans-hall"
data = hc.do_query(thermostat_query % BUILDING)["Rows"]

In [140]:
print(data)

[{'?zone': 'HVAC_Zone_AC-6', '?uri': 'avenal-veterans-hall/devices/s.pelican/AC-6/i.xbos.thermostat'}, {'?zone': 'HVAC_Zone_AC-5', '?uri': 'avenal-veterans-hall/devices/s.pelican/AC-5/i.xbos.thermostat'}, {'?zone': 'HVAC_Zone_AC-4', '?uri': 'avenal-veterans-hall/devices/s.pelican/AC-4/i.xbos.thermostat'}, {'?zone': 'HVAC_Zone_AC-3', '?uri': 'avenal-veterans-hall/devices/s.pelican/AC-3/i.xbos.thermostat'}, {'?zone': 'HVAC_Zone_AC-2', '?uri': 'avenal-veterans-hall/devices/s.pelican/AC-2/i.xbos.thermostat'}, {'?zone': 'HVAC_Zone_AC-1', '?uri': 'avenal-veterans-hall/devices/s.pelican/AC-1/i.xbos.thermostat'}]


In [141]:
tstats = {d["?zone"]: Thermostat(client, d["?uri"]) for d in data}

Got Thermostat at avenal-veterans-hall/devices/s.pelican/AC-6/i.xbos.thermostat last alive 2018-05-29T18:59:43.147636514Z
Got Thermostat at avenal-veterans-hall/devices/s.pelican/AC-5/i.xbos.thermostat last alive 2018-05-29T18:59:43.14379392Z
Got Thermostat at avenal-veterans-hall/devices/s.pelican/AC-4/i.xbos.thermostat last alive 2018-05-29T18:59:43.140221938Z
Got Thermostat at avenal-veterans-hall/devices/s.pelican/AC-3/i.xbos.thermostat last alive 2018-05-29T18:59:43.136547071Z
Got Thermostat at avenal-veterans-hall/devices/s.pelican/AC-2/i.xbos.thermostat last alive 2018-05-29T18:59:43.132875511Z
Got Thermostat at avenal-veterans-hall/devices/s.pelican/AC-1/i.xbos.thermostat last alive 2018-05-29T18:59:43.128491977Z


In [143]:
f_tstat = tstats["HVAC_Zone_AC-4"]
f_tstat._uri

'avenal-veterans-hall/devices/s.pelican/AC-4/i.xbos.thermostat'

In [144]:
COOLING_ACTION = lambda tstat: {"heating_setpoint": 50, "cooling_setpoint": 65, "override": True,
                                "mode": 3}
HEATING_ACTION = lambda tstat: {"heating_setpoint": 80, "cooling_setpoint": 95, "override": True,
                                "mode": 3}
NO_ACTION = lambda tstat: {"heating_setpoint": tstat.temperature - 5,
                                "cooling_setpoint": tstat.temperature + 5,
                                        "override": True, "mode": 3}
print(COOLING_ACTION(f_tstat))

{'override': True, 'cooling_setpoint': 65, 'mode': 3, 'heating_setpoint': 50}


In [149]:
f_tstat.write(NO_ACTION(f_tstat))

In [150]:
def printData(tstat):
    print("heating setpoint", tstat.heating_setpoint)
    print("cooling setpoint", tstat.cooling_setpoint)
    print("temperature", tstat.temperature)

In [154]:
for zone, tstat in tstats.items():
    print(zone)
    print(tstat._uri)
    printData(tstat)

HVAC_Zone_AC-6
avenal-veterans-hall/devices/s.pelican/AC-6/i.xbos.thermostat
('heating setpoint', 68.0)
('cooling setpoint', 78.0)
('temperature', 75.0)
HVAC_Zone_AC-5
avenal-veterans-hall/devices/s.pelican/AC-5/i.xbos.thermostat
('heating setpoint', 69.0)
('cooling setpoint', 74.0)
('temperature', 73.9)
HVAC_Zone_AC-4
avenal-veterans-hall/devices/s.pelican/AC-4/i.xbos.thermostat
('heating setpoint', 70.0)
('cooling setpoint', 73.0)
('temperature', 73.4)
HVAC_Zone_AC-3
avenal-veterans-hall/devices/s.pelican/AC-3/i.xbos.thermostat
('heating setpoint', 70.0)
('cooling setpoint', 73.0)
('temperature', 73.3)
HVAC_Zone_AC-2
avenal-veterans-hall/devices/s.pelican/AC-2/i.xbos.thermostat
('heating setpoint', 70.0)
('cooling setpoint', 73.0)
('temperature', 73.3)
HVAC_Zone_AC-1
avenal-veterans-hall/devices/s.pelican/AC-1/i.xbos.thermostat
('heating setpoint', 70.0)
('cooling setpoint', 73.0)
('temperature', 73.0)
